# Sequence Analysis with Python

Contact: Veli Mäkinen veli.makinen@helsinki.fi

The following assignments introduce applications of hashing with ```dict()``` primitive of Python. While doing so, a rudimentary introduction to biological sequences is given.
This framework is then enhanced with probabilities, leading to routines to generate random sequences under some constraints, including a general concept of *Markov-chains*. All these components illustrate the usage of ```dict()```, but at the same time introduce some other computational routines to efficiently deal with probabilities.   
The function ```collections.defaultdict``` can be useful.

Below are some "suggested" imports. Feel free to use and modify these, or not. Generally it's good practice to keep most or all imports in one place. Typically very close to the start of notebooks.

In [1]:
import numpy as np
from collections import defaultdict
from itertools import product

import random
from numpy.random import choice

import requests
import re

The automated TMC tests do not test cell outputs. These are intended to be evaluated in the peer reviews. So it is still be a good idea to make the outputs as clear and informative as possible.

To keep TMC tests running as well as possible it is recommended to keep global variable assignments in the notebook to a minimum to avoid potential name clashes and confusion. Additionally you should keep all actual code exection in main guards to keep the test running smoothly. If you run [check_sequence.py](https://raw.githubusercontent.com/saskeli/data-analysis-with-python-summer-2019/master/check_outputs.py) in the `part07-e01_sequence_analysis` folder, the script should finish very quickly and optimally produce no output.

If you download data from the internet during execution (codon usage table), the parts where downloading is done should not work if you decide to submit to the tmc server. Local tests should work fine.

## DNA and RNA

A DNA molecule consist, in principle, of a chain of smaller molecules. These smaller molecules have some common basic components (bases) that repeat. For our purposes it is sufficient to know that these bases are nucleotides adenine, cytosine, guanine, and thymine with abbreviations ```A```, ```C```, ```G```, and ```T```. Given a *DNA sequence* e.g. ```ACGATGAGGCTCAT```, one can reverse engineer (with negligible loss of information) the corresponding DNA molecule.

Parts of a DNA molecule can *transcribe* into an RNA molecule. In this process, thymine gets replaced by uracil (```U```).


1. Write a function ```dna_to_rna``` to convert a given DNA sequence $s$ into an RNA sequence. For the sake of exercise, use ```dict()``` to store the symbol to symbol encoding rules. Create a program to test your function.

In [2]:
def dna_to_rna(s):
    '''Transcribe a DNA string into an RNA string'''
    return s.replace('T', 'U')
    
if __name__ == '__main__':
    print(dna_to_rna('AACGTGATTTC'))

AACGUGAUUUC


### Idea of solution

To convert a DNA sequence into its corresponding RNA sequence, thymine (T) is substituted with uracil (U) using the Python method `replace()`.

### Discussion

The solution performs precisely as expeted. In molecular biology, RNA is produced from DNA through a process known as transcription. This transformation is crucial for protein synthesis, as RNA acts as the template that translates genetic instructions into proteins, which are composed of their fundamental units, amino acids.

## Proteins

Like DNA and RNA, protein molecule can be interpreted as a chain of smaller molecules, where the bases are now amino acids. RNA molecule may *translate* into a protein molecule, but instead of base by base, three bases of RNA correspond to one base of protein. That is, RNA sequence is read triplet (called codon) at a time.

2. Consider the codon to amino acid conversion table in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html. Write a function ```get_dict``` to read the table into a ```dict()```, such that for each RNA sequence of length 3, say $\texttt{AGU}$, the hash table stores the conversion rule to the corresponding amino acid. You may store the html page to your local src directory,
and parse that file.

In [3]:
def get_table():
    ''' Fetch "Codon Usage Table" either from file or URL '''
    try:
        with open('src/codon_usage_table.html', 'r') as file:
            html_content = file.read()
    except:
        url = 'https://raw.githubusercontent.com/csmastersUH/data_analysis_with_python_2020/master/Codon%20usage%20table.html'
        response = requests.get(url)
        html_content = response.text
    return html_content

def get_dict():
    '''
    Parse Codon Usage Table into Codon to Aminoacid table.
    '''
    html_content = get_table()
    # Find the <pre> tag content
    pre_content = re.search(r'<pre>(.*?)</pre>', html_content, re.DOTALL)
    if pre_content:
        codon_dict = {}
        # Find all codon lines in <pre> tag to capture the codon and its corresponding amino acid
        pattern = r'([AUGC]{3})\s([A-Z*])'
        matches = re.findall(pattern, html_content)
        for codon, amino_acid in matches:
            codon_dict[codon] = amino_acid
        return codon_dict
    return {}

if __name__ == '__main__':
    codon_to_aa = get_dict()
    print(codon_to_aa)

{'UUU': 'F', 'UCU': 'S', 'UAU': 'Y', 'UGU': 'C', 'UUC': 'F', 'UCC': 'S', 'UAC': 'Y', 'UGC': 'C', 'UUA': 'L', 'UCA': 'S', 'UAA': '*', 'UGA': '*', 'UUG': 'L', 'UCG': 'S', 'UAG': '*', 'UGG': 'W', 'CUU': 'L', 'CCU': 'P', 'CAU': 'H', 'CGU': 'R', 'CUC': 'L', 'CCC': 'P', 'CAC': 'H', 'CGC': 'R', 'CUA': 'L', 'CCA': 'P', 'CAA': 'Q', 'CGA': 'R', 'CUG': 'L', 'CCG': 'P', 'CAG': 'Q', 'CGG': 'R', 'AUU': 'I', 'ACU': 'T', 'AAU': 'N', 'AGU': 'S', 'AUC': 'I', 'ACC': 'T', 'AAC': 'N', 'AGC': 'S', 'AUA': 'I', 'ACA': 'T', 'AAA': 'K', 'AGA': 'R', 'AUG': 'M', 'ACG': 'T', 'AAG': 'K', 'AGG': 'R', 'GUU': 'V', 'GCU': 'A', 'GAU': 'D', 'GGU': 'G', 'GUC': 'V', 'GCC': 'A', 'GAC': 'D', 'GGC': 'G', 'GUA': 'V', 'GCA': 'A', 'GAA': 'E', 'GGA': 'G', 'GUG': 'V', 'GCG': 'A', 'GAG': 'E', 'GGG': 'G'}


### Idea of solution

The `get_table()` function tries to retrieve the `Codon Usage Table` from a local file. If it is fail, the function will fetche the table from the given URL. Since the retrieved data is in HTML format, the `get_table()` function is utilized to generate a dictionary that maps each codon to its corresponding amino acid.

### Discussion

The output works as expected, extracting the relevant data from the Codon Usage Table. This table illustrates the frequency with which specific codons are utilized in the coding sequences of genes across various organisms. Additionally, the mapping of codons to amino acids is fundamental in molecular biology, as it enables researchers to interpret genetic sequences and predict the resulting protein structure.

**Note 1:** This notebook will focus exclusively on the provided Codon Usage Table, although various such tables can be retrieved from the Internet (e.g. https://www.bioinformatics.org/sms2/codon_usage.html)

**Note 2:** While a Pandas DataFrame could be a good prospective data structure, utilizing dictionaries will be sufficient.

3. Use the same conversion table as above, but now write function `get_dict_list` to read the table into a `dict()`, such that for each amino acid the hash table stores the list of codons encoding it.    

In [4]:
def get_dict_list():
    '''
    Map each amino acid to a list of corresponding codons.
    {'*': ['UAA', 'UGA', 'UAG'], 'A': ['GCU', 'GCC', 'GCA', 'GCG'], ...}
    '''
    codon_to_aa = get_dict()
    # Create lists for new keys
    aa_to_codon = defaultdict(list)
    [aa_to_codon[amino_acid].append(codon) for codon, amino_acid in codon_to_aa.items()]
    # Convert defaultdict back to a regular dict
    return dict(aa_to_codon)

if __name__ == '__main__':
    aa_to_codons = get_dict_list()
    print(aa_to_codons)

{'F': ['UUU', 'UUC'], 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], 'Y': ['UAU', 'UAC'], 'C': ['UGU', 'UGC'], 'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'], '*': ['UAA', 'UGA', 'UAG'], 'W': ['UGG'], 'P': ['CCU', 'CCC', 'CCA', 'CCG'], 'H': ['CAU', 'CAC'], 'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'Q': ['CAA', 'CAG'], 'I': ['AUU', 'AUC', 'AUA'], 'T': ['ACU', 'ACC', 'ACA', 'ACG'], 'N': ['AAU', 'AAC'], 'K': ['AAA', 'AAG'], 'M': ['AUG'], 'V': ['GUU', 'GUC', 'GUA', 'GUG'], 'A': ['GCU', 'GCC', 'GCA', 'GCG'], 'D': ['GAU', 'GAC'], 'G': ['GGU', 'GGC', 'GGA', 'GGG'], 'E': ['GAA', 'GAG']}


### Idea of solution

The `get_dict_list()` function generates a dictionary that maps each amino acid to a list of corresponding codons. It utilizes the `defaultdict` collection to enhance the efficiency of maintaining counts of these mappings. This function will serve as the backbone for most of the code in this notebook.

### Discussion

The output works as expected, mapping each aminoacid to its corresponding set of codons. This process takes into account the frequency of codon usage in different organisms, which can significantly affect protein expression. In theory, this mapping can be valuable for reverse genetic modifications, where researchers may need to convert amino acids back to potential codons. 

With the conversion tables at hand, the following should be trivial to solve.

4. Fill in function ```rna_to_prot``` in the stub solution to convert a given DNA sequence $s$ into a protein sequence.
You may use the dictionaries from exercises 2 and 3. You can test your program with `ATGATATCATCGACGATGTAG`.

In [5]:
def rna_to_prot(s):
    ''' Returns the corresponding amino acid string from an RNA string '''
    protein = ''
    # Get codon to amino acid mapping
    codon_dict = get_dict()
    # Use list comprehension to build the protein string
    protein = ''.join(codon_dict[s[i:i + 3]] for i in range(0, len(s), 3) if s[i:i + 3] in codon_dict)
    return protein

def dna_to_prot(s):
    ''' Returns DNA to amino acid string.'''
    return rna_to_prot(dna_to_rna(s))

if __name__ == '__main__':
    print(dna_to_prot('ATGATATCATCGACGATGTAG'))

MISSTM*


### Idea of solution

The `rna_to_prot()` function transcribes an RNA sequence into its corresponding protein sequence by translating each codon (triplet of nucleotides) into its respective amino acid. The `dna_to_prot()` function first converts a DNA sequence to RNA and then translates that RNA into a protein sequence, utilizing the functions that have been defined previously.

### Discussion

The output works as expected, including the stop codons. This two-step conversion reflects the central dogma of molecular biology: **DNA → RNA → Protein**. It encapsulates the essential processes of transcription and translation, enabling the analysis of entire gene sequences to predict the resulting proteins.

You may notice that there are $4^3=64$ different codons, but only 20 amino acids. That is, some triplets encode the same amino acid.  

## Reverse translation

It has been observed that among the codons coding the same amino acid, some are more frequent than others. These frequencies can be converted to probabilities. E.g. consider codons `AUU`, `AUC`, and `AUA` that code for amino acid isoleucine.
If they are observed, say, 36, 47, 17 times, respectively, to code isoleucine in a dataset, the probability that a random such event is `AUU` $\to$ isoleucine is 36/100.

This phenomenon is called *codon adaptation*, and for our purposes it works as a good introduction to generation of random sequences under constraints.   

5. Consider the codon adaptation frequencies in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html and read them into a ```dict()```, such that for each RNA sequence of length 3, say `AGU`, the hash table stores the probability of that codon among codons encoding the same amino acid.
Put your solution in the ```get_probabability_dict``` function. Use the column "([number])" to estimate the probabilities, as the two preceding columns contain truncated values.  

In [6]:
def get_probability_dict():
    '''
    Fetches probabilities for all codons with respect to their amino acid into a dictionary.
    {'AAA': 0.434049, 'AAC': 0.529633, 'AAG': 0.565951, ...}
    '''
    html_content = get_table()
    pre_content = re.search(r'<pre>(.*?)</pre>', html_content, re.DOTALL)  # Find all codon lines in the <pre> tag
    if pre_content:
        pre_data = pre_content.group(1)
        # Initialize dictionaries to hold counts and probabilities
        codon_dict = get_dict()
        codon_counts = defaultdict(int)
        amino_acid_counts = defaultdict(int)
        # Find all matches for codons, amino acids, and counts
        pattern = r'([AUGC]{3})\s+([A-Z\*])\s+.*?\s*\(\s*(\d+)\s*\)'
        matches = re.findall(pattern, pre_data)
        # Calculate the counts
        for codon, amino_acid, count in matches:
            count = int(count)  
            codon_counts[codon] += count
            amino_acid_counts[amino_acid] += count
        # Calculate probabilities
        probability_dict = {}
        for codon, count in codon_counts.items():
            probability = count / amino_acid_counts[codon_dict[codon]]
            probability_dict[codon] = probability
        return probability_dict
    return {}

if __name__ == '__main__':
    codon_to_prob = get_probability_dict()
    items = sorted(codon_to_prob.items(), key=lambda x: x[0])
    for i in range(1 + len(items)//6):
        print('\t'.join(
            f'{k}: {v:.6f}'
            for k, v in items[i*6:6+i*6]
        ))

AAA: 0.434049	AAC: 0.529633	AAG: 0.565951	AAU: 0.470367	ACA: 0.284188	ACC: 0.355232
ACG: 0.113812	ACU: 0.246769	AGA: 0.214658	AGC: 0.239938	AGG: 0.211091	AGU: 0.149602
AUA: 0.169062	AUC: 0.469866	AUG: 1.000000	AUU: 0.361072	CAA: 0.265017	CAC: 0.581485
CAG: 0.734983	CAU: 0.418515	CCA: 0.276603	CCC: 0.323470	CCG: 0.113196	CCU: 0.286731
CGA: 0.108812	CGC: 0.183777	CGG: 0.201554	CGU: 0.080108	CUA: 0.071380	CUC: 0.195577
CUG: 0.395702	CUU: 0.131716	GAA: 0.422453	GAC: 0.535458	GAG: 0.577547	GAU: 0.464542
GCA: 0.228121	GCC: 0.399781	GCG: 0.106176	GCU: 0.265922	GGA: 0.249922	GGC: 0.337109
GGG: 0.249882	GGU: 0.163087	GUA: 0.116577	GUC: 0.238306	GUG: 0.463346	GUU: 0.181770
UAA: 0.297019	UAC: 0.556662	UAG: 0.236738	UAU: 0.443338	UCA: 0.150517	UCC: 0.217960
UCG: 0.054398	UCU: 0.187586	UGA: 0.466243	UGC: 0.543843	UGG: 1.000000	UGU: 0.456157
UUA: 0.076568	UUC: 0.535866	UUG: 0.129058	UUU: 0.464134


### Idea of solution

The `get_probability_dict()` function computes the probability of each codon based on counts extracted from the Codon Usage Table. It leverages previous building blocks for fetching the Codon Usage Table, parsing the HTML data into numeric data, and then performing **counting** and **probability computing**.

### Discussion

The output works as expected: it is a clean table of probabilities for each codon separate from their corresponding amino acid that determines the probability distributions. By selecting codons that are most likely to be translated efficiently in a given organism, this knowledge will enhance protein expression for subsequent function definitions in this notebook.

Now you should have everything in place to easily solve the following.


6. Write a class ```ProteinToMaxRNA``` with a ```convert``` method which converts a protein sequence into the most likely RNA sequence to be the source of this protein. Run your program with `LTPIQNRA`.

In [7]:
class ProteinToMaxRNA:
    ''' Maps a protein sequence into its most likely RNA sequence. '''
    def __init__(self):
        pass

    def convert(self, s):
        ''' 'LTPIQNRA' -> 'CUGACCCCCAUCCAGAACAGAGCC' '''
        # Get amino acid to codon and probability for all codons dictionaries
        protein_map = get_dict_list()
        codon_probability = get_probability_dict()
        RNA = []
        for protein in s:
            codons = protein_map[protein]
            # Select the codon with the maximum probability
            max_codon = max(codons, key=lambda codon: codon_probability.get(codon, 0))
            RNA.append(max_codon)
        return ''.join(RNA)

if __name__ == '__main__':
    protein_to_rna = ProteinToMaxRNA()
    print(protein_to_rna.convert('LTPIQNRA'))

CUGACCCCCAUCCAGAACAGAGCC


### Idea of solution

This class and method generate the most probable RNA sequence for a given protein using codon probabilities. While the probabilities can be utilized for better data wrangling, for now, it suffices to select the codon with the maximum probability. The `max()` method, along with a lambda function, proves useful for extracting the maximum probability from the codon dictionary.

### Discussion

Because of its limited scope (max sampling), the output sequence is expected to be random yet with a low degree of variance. This approach will not be used anymore in this notebook.

Now we are almost ready to produce random RNA sequences that code a given protein sequence. For this, we need a subroutine to *sample from a probability distribution*. Consider our earlier example of probabilities 36/100, 47/100, and 17/100 for `AUU`, `AUC`, and `AUA`, respectively.
Let us assume we have a random number generator ```random()``` that returns a random number from interval $[0,1)$. We may then partition the unit interval according to cumulative probabilities to $[0,36/100), [36/100,83/100), [83/100,1)$, respectively. Depending which interval the number ```random()``` hits, we select the codon accordingly.

7. Write a function ```random_event``` that chooses a random event, given a probability distribution (set of events whose probabilities sum to 1).
You can use function ```random.uniform``` to produce values uniformly at random from the range $[0,1)$. The distribution should be given to your function as a dictionary from events to their probabilities.

In [8]:
import random

def random_event(dist):
    '''
    Takes as input a dictionary from events to their probabilities.
    Returns a random event sampled according to the given distribution.
    The probabilities must sum to 1.0
    Ex: T, A, G, T, C, C, T, C, etc.
    '''
    events, weights = zip(*dist.items())
    return random.choices(events, weights=weights)[0]

if __name__ == '__main__':
    distribution = dict(zip('ACGT', [0.10, 0.35, 0.15, 0.40]))
    print(', '.join(random_event(distribution) for _ in range(29)))

C, T, A, G, C, T, C, G, T, A, G, C, T, T, A, T, A, C, C, T, T, T, G, G, T, G, C, A, T


### Idea of solution

The `random_event()` function randomly selects an event (a nucleotide) based on a given probability distribution. It efficiently utilizes the Python `random` library to sample uniformly using the weights provided through the distribution dictionary.

### Discussion

The `random_event()` function forms the basis for creating probabilistic models that will be further explored in this notebook. Random sampling is essential in computational biology for simulating biological processes, such as mutation rates or gene expression.

With this general routine, the following should be easy to solve.

8. Write a class ```ProteinToRandomRNA``` to produce a random RNA sequence encoding the input protein sequence according to the input codon adaptation probabilities. The actual conversion is done through the ```convert``` method. Run your program with `LTPIQNRA`.

In [9]:
class ProteinToRandomRNA:
    ''' Maps a protein sequence into a random RNA sequence based on probabilities. '''
    def __init__(self):
        pass

    def convert(self, s):
        ''' "LTPIQNRA" -> "CUGACUCCUAUCCAGAACCGGGCC" '''
        # Get amino acid to codon and probability for all codons dictionaries
        protein_map = get_dict_list()
        codon_probability = get_probability_dict()
        RNA = []
        for protein in s:
            codons = protein_map[protein]
            # Create a probability distribution for the codons
            probability_distribution = [codon_probability[codon] for codon in codons]
            RNA.append(random_event(dict(zip(codons, probability_distribution))))
        return ''.join(RNA)

if __name__ == '__main__':
    protein_to_random_codons = ProteinToRandomRNA()
    print(protein_to_random_codons.convert('LTPIQNRA'))

CUGACCCCCAUCCAGAAUAGGGCC


### Idea of solution

This class and method generate a random RNA sequence for a given protein, taking into account `Codon Usage Probabilities` and their extracted distributions of each amino acid. 

### Discussion

The output works as expected with no clear sequential relationships yet (which will be provided by Markov Chains). Even with this limitation, through the lens of probability, we can now explore the generative potential of having genetic data. This allows us to create diverse synthetic RNA sequences based on specific protein sequences, which are crucial in real-life scenarios such as genetic engineering.

## Generating DNA sequences with higher-order Markov chains

We will now reuse the machinery derived above in a related context. We go back to DNA sequences, and consider some easy statistics that can be used to characterize the sequences.
First, just the frequencies of bases $\texttt{A}$, $\texttt{C}$, $\texttt{G}$, $\texttt{T}$ may reveal the species from which the input DNA originates; each species has a different base composition that has been formed during evolution.
More interestingly, the areas where DNA to RNA transcription takes place (coding region) have an excess of $\texttt{C}$ and $\texttt{G}$ over $\texttt{A}$ and $\texttt{T}$. To detect such areas a common routine is to just use a *sliding window* of fixed size, say $k$, and compute for each window position
$T[i..i+k-1]$ the base frequencies, where $T[1..n]$ is the input DNA sequence. When sliding the window from  $T[i..i+k-1]$ to $T[i+1..i+k]$ frequency $f(T[i])$ gets decreases by one and $f(T[i+k])$ gets increased by one.

9. Write a *generator* ```sliding_window``` to compute sliding window base frequencies so that each moving of the window takes constant time. We saw in the beginning of the course one way how to create generators using
  generator expression. Here we use a different way. For the function ```sliding_window``` to be a generator, it must have at least   one ```yield``` expression, see [https://docs.python.org/3/reference/expressions.html#yieldexpr](https://docs.python.org/3/reference/expressions.html#yieldexpr).
  
  Here is an example of a generator expression that works similarily to the built in `range` generator:
  ```Python
  def range(a, b=None, c=1):
      current = 0 if b == None else a
      end = a if b == None else b
      while current < end:
          yield current
          current += c
  ```
  A yield expression can be used to return a value and *temporarily* return from the function.

In [10]:
def sliding_window(s, k):
    '''
    Returns a generator that can be iterated over
    starting positions of a k-window in the sequence.
    Each starting position yields the nucleotide frequencies in the window as a dictionary.
    {'A': 0, 'C': 3, 'G': 0, 'T': 1}
    ...
    {'A': 0, 'C': 2, 'G': 1, 'T': 1}
    '''
    nucleotide_freqs = {nucleotide: 0 for nucleotide in 'ACGT'}
    # Initialize the frequency count for the first window
    for nucleotide in s[:k]:
        nucleotide_freqs[nucleotide] += 1

    yield nucleotide_freqs

    # Slide the window across the sequence
    for i in range(1, len(s) - k + 1):
        nucleotide_freqs[s[i - 1]] -= 1  # Remove count of the outgoing nucleotide
        nucleotide_freqs[s[i + k - 1]] += 1  # Add count of the incoming nucleotide
        yield nucleotide_freqs  # Yield the updated frequencies

if __name__ == '__main__':
    s = 'TCCCGACGGCCTTGCC'
    for d in sliding_window(s, 4):
        print(d)

{'A': 0, 'C': 3, 'G': 0, 'T': 1}
{'A': 0, 'C': 3, 'G': 1, 'T': 0}
{'A': 1, 'C': 2, 'G': 1, 'T': 0}
{'A': 1, 'C': 2, 'G': 1, 'T': 0}
{'A': 1, 'C': 1, 'G': 2, 'T': 0}
{'A': 1, 'C': 1, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 1, 'T': 1}
{'A': 0, 'C': 2, 'G': 0, 'T': 2}
{'A': 0, 'C': 1, 'G': 1, 'T': 2}
{'A': 0, 'C': 1, 'G': 1, 'T': 2}
{'A': 0, 'C': 2, 'G': 1, 'T': 1}


### Idea of solution

The `sliding_window()` function yields a temporary value that provides nucleotide frequencies for overlapping windows of length **k** across the sequence. It leverages the optimal capabilities of generator expressions, allowing it to handle sequences of arbitrary length and at an arbitrary phase without consuming as much memory as conventional list processing.

### Discussion

The output dictionaries work as expected. While the `sliding_window()` function will not be further utilized in this notebook, it introduces a more sophisticated approach for analyzing genetic data, which is inherently sequential. Sliding windows are a common approach in sequence analysis, particularly in genomics, but also in all sorts of time-series and signal-processing duties. In our particular context, examining nucleotide frequencies can help identify regions of significance, such as motifs or other important sequence features.


Our models so far have been so-called *zero-order* models, as each event has been independent of other events. With sequences, the dependencies of events are naturally encoded by their *contexts*. Considering that a sequence is produced from left-to-right, a *first-order* context for $T[i]$ is $T[i-1]$, that is, the immediately preceding symbol. *First-order Markov chain* is a sequence produced by generating $c=T[i]$ with the probability of event of seeing symbol $c$ after previously generated symbol $a=T[i-1]$. The first symbol of the chain is sampled according to the zero-order model.  
The first-order model can naturally be extended to contexts of length $k$, with $T[i]$ depending on $T[i-k..i-1]$. Then the first $k$ symbols of the chain are sampled according to the zero-order model.  The following assignments develop the routines to work with the *higher-order Markov chains*.
In what follows, a $k$-mer is a substring $T[i..i+k-1]$ of the sequence at an arbitrary position.

10. Write function ```context_list``` that given an input DNA sequence $T$ associates to each $k$-mer $W$ the concatenation of all symbols $c$ that appear after context $W$ in $T$, that is, $T[i..i+k]=Wc$. For example, <span style="color:red; font:courier;">GA</span> is associated to <span style="color:blue; font: courier;">TCT</span> in $T$=<span style="font: courier;">AT<span style="color:red;">GA</span><span style="color:blue;">T</span>ATCATC<span style="color:red;">GA</span><span style="color:blue;">C</span><span style="color:red;">GA</span><span style="color:blue;">T</span>GTAG</span>, when $k=2$.

In [11]:
def context_list(s, k):
    '''
    Returns a dictionary mapping k-mers to their following nucleotides.
    {'AC': 'G', 'AT': 'GACCC', ..., }
    '''
    wc = defaultdict(list)
    if k == 0:
        return {'': s}
    # Iterate through the string, stopping early enough to avoid index errors
    for i in range(len(s) - k):
        current_kmer = s[i:i + k]
        following_symbol = s[i + k]
        wc[current_kmer].append(following_symbol)

    return {key: ''.join(value) for key, value in wc.items()}

if __name__ == '__main__':
    k = 2
    s = 'ATGATATCATCGACGATCTAG'
    d = context_list(s, k)
    print(d)

{'AT': 'GACCC', 'TG': 'A', 'GA': 'TCT', 'TA': 'TG', 'TC': 'AGT', 'CA': 'T', 'CG': 'AA', 'AC': 'G', 'CT': 'A'}


### Idea of solution

The `context_list()` function creates a mapping of k-mers to the nucleotides that immediately follow them in the sequence. It utilizes the capabilities of the `defaultdict` object to efficiently keep track of k-mers and their corresponding following nucleotides.

### Discussion

The resulting dictionary works as expected, providing the foundations for context-based models for sequence analysis, and enabling the prediction of nucleotide behavior based on preceding sequences. In this simple exercise using genomic data, we can gain insight into how sequences are formed in a manner that is not stochastic but probabilistic.

11. With the above solution, write function ```context_probabilities``` to count the frequencies of symbols in each context and convert these frequencies into probabilities. Run `context_probabilities` with $T=$ `ATGATATCATCGACGATGTAG` and $k$ values 0 and 2.

In [12]:
def context_probabilities(s, k):
    '''
    Computes probabilities of nucleotides following each k-mer in the sequence.
    '''
    if k == 0:  # Resulting context is equal to input sequence
        return {'': {nucleotide: s.count(nucleotide) / len(s) for nucleotide in 'ACGT'}}

    context = context_list(s, k)
    probabilities = {}
    kmer_freqs = {}
    for kmer, follows in context.items():
        total_follows = len(follows)
        for nucleotide in 'ACGT':
            kmer_freqs[nucleotide] = follows.count(nucleotide)
        probabilities[kmer] = {nucleotide: count / total_follows for nucleotide, count in kmer_freqs.items()}

    return probabilities

if __name__ == '__main__':
    s = 'ATGATATCATCGACGATCTAG'
    print(context_probabilities(s, 0))
    print(context_probabilities(s, 2))

{'': {'A': 0.3333333333333333, 'C': 0.19047619047619047, 'G': 0.19047619047619047, 'T': 0.2857142857142857}}
{'AT': {'A': 0.2, 'C': 0.6, 'G': 0.2, 'T': 0.0}, 'TG': {'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0}, 'GA': {'A': 0.0, 'C': 0.3333333333333333, 'G': 0.0, 'T': 0.6666666666666666}, 'TA': {'A': 0.0, 'C': 0.0, 'G': 0.5, 'T': 0.5}, 'TC': {'A': 0.3333333333333333, 'C': 0.0, 'G': 0.3333333333333333, 'T': 0.3333333333333333}, 'CA': {'A': 0.0, 'C': 0.0, 'G': 0.0, 'T': 1.0}, 'CG': {'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0}, 'AC': {'A': 0.0, 'C': 0.0, 'G': 1.0, 'T': 0.0}, 'CT': {'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0}}


### Idea of solution

The `context_probabilities()` function calculates the probabilities of each nucleotide following each k-mer within the sequence. It essentially utilizes the contextual frequency analysis facilitated by the preceding `context_list()` function to derive these probabilities.

### Discussion

The output dictionary correctly maps the probabilistic relationships to the provided input sequences. Understanding the conditional probabilities of nucleotide occurrences enhances our grasp of sequence dynamics and facilitates the development of sequence models, such as `Markov Chains`, which can illustrate the inheritance of genetic traits. This, in turn, is important because sequential data implies causal relationships that can be explored in increasingly complex contexts, along with their associated probability distributions.

12. With the above solution and the function ```random_event``` from the earlier exercise, write class ```MarkovChain```. Its ```generate``` method should generate a random DNA sequence following the original $k$-th order Markov chain probabilities.

In [13]:
class MarkovChain:
    ''' Generates sequences based on a Markov Chain model'''
    def __init__(self, zeroth, kth, k=2):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth

    def generate(self, n, seed=None):
        ''' Generate a random sequence of length n using the Markov Chain model '''
        if n == 0:
            return []

        if seed is not None:
            random.seed(seed)

        start = random_event(self.zeroth)
        sequence = [start]

        while n > len(sequence):
            step = ''.join(sequence[-self.k:])
            if step in self.kth:
                next_nucleotide = random_event(self.kth[step])
            else:
                next_nucleotide = random_event(self.zeroth)
            sequence.append(next_nucleotide)

        return ''.join(sequence)

if __name__ == '__main__':
    zeroth = {'A': 0.2, 'C': 0.19, 'T': 0.31, 'G': 0.3}
    kth = {'GT': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0},
           'CA': {'A': 0.0, 'C': 0.0, 'T': 1.0, 'G': 0.0},
           'TC': {'A': 0.5, 'C': 0.0, 'T': 0.0, 'G': 0.5},
           'GA': {'A': 0.0, 'C': 0.3333333333333333, 'T': 0.6666666666666666, 'G': 0.0},
           'TG': {'A': 0.5, 'C': 0.0, 'T': 0.5, 'G': 0.0},
           'AT': {'A': 0.2, 'C': 0.4, 'T': 0.0, 'G': 0.4},
           'TA': {'A': 0.0, 'C': 0.0, 'T': 0.5, 'G': 0.5},
           'AC': {'A': 0.0, 'C': 0.0, 'T': 0.0, 'G': 1.0},
           'CG': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}}
    n = 10
    seed = 0
    mc = MarkovChain(zeroth, kth)
    print(mc.generate(n, seed))

GGTAGTATCG


### Idea of solution

This class and method generate random nucleotide sequences using a `Markov Chains` based on given zeroth and k-th probabilities. It effectively utilizes previously defined functions to create sequences that are no longer uniformly random, providing more coherent contextual information. This process is facilitated by the combined use of random event sampling and the pre-defined zeroth and k-th probability distributions.

### Discussion

The sequence exhibits the usefulness of `Markov Chains` as modeling tools for computational biology. In theory, this approach can simulate how genes evolve over generations and can be used to model various aspects of genetic similarities and variations across species. Moreover, this implementation of `Markov Chains` signals their utility beyond the biological domain, serving as generative tools for a wide range of applications.

If you have survived so far without problems, please run your program a few more times with different inputs. At some point you should get a lookup error in your hash-table! The reason for this is not your code, but the way we defined the model: Some $k$-mers may not be among the training data (input sequence $T$), but such can be generated as the first $k$-mer that is generated using the zero-order model.  

A general approach to fixing such issues with incomplete training data is to use *pseudo counts*. That is, all imaginable events are initialized to frequency count 1.   

13. Write a new solution `context_pseudo_probabilities` based on the solution to problem 11. But this time use pseudo counts in order to obtain a $k$-th order Markov chain that can assign a probability for any DNA sequence. You may use the standard library function `itertools.product` to iterate over all $k$-mer of given length (`product("ACGT", repeat=k)`).

In [14]:
def context_pseudo_probabilities(s, k):
    '''
    Computes pseudo probabilities of nucleotides following each k-mer.
    '''
    # To count occurrences of k-mers and their following nucleotides
    context = context_list(s, k)
    probabilities = {}
    # Generate all possible k-mers of length k
    all_kmers = [''.join(kmer) for kmer in product('ACGT', repeat=k)]
    for kmer in all_kmers:
        if kmer in context:
            # '' is fallback in case no key is found (otherwise known as zeroth)
            follows = context.get(kmer, '')
            # Add pseudo count for each nucleotide base 'ACGT'
            total_follows = len(follows) + 4
            # Count occurrences of nucleotides that follow the k-mer, add pseudo-count then calculate probabilities
            probabilities[kmer] = {nucleotide: (follows.count(nucleotide) + 1) / total_follows for nucleotide in 'ACGT'}
        else:
            probabilities[kmer] = {nucleotide: 0.25 for nucleotide in 'ACGT'}

    return probabilities

if __name__ == '__main__':
    k = 2
    s = 'ATGATATCATCGACGATGTAG'
    kth = context_pseudo_probabilities(s, k)
    zeroth = context_pseudo_probabilities(s, 0)['']
    print(f'zeroth: {zeroth}')
    print('\n'.join(f'{k}: {dict(v)}' for k, v in kth.items()))

    print('\n', MarkovChain(zeroth, kth, k).generate(20))

zeroth: {'A': 0.32, 'C': 0.16, 'G': 0.24, 'T': 0.28}
AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AC: {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2}
AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AT: {'A': 0.2222222222222222, 'C': 0.3333333333333333, 'G': 0.3333333333333333, 'T': 0.1111111111111111}
CA: {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4}
CC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CG: {'A': 0.5, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.16666666666666666}
CT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GA: {'A': 0.14285714285714285, 'C': 0.2857142857142857, 'G': 0.14285714285714285, 'T': 0.42857142857142855}
GC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GT: {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2}
TA: {'A': 0.16666666666666666, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.3333333333333333}
TC: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.16666666666666666

### Idea of solution

The `context_pseudo_probabilities()` function computes pseudo probabilities for nucleotides that follow each k-mer, incorporating pseudo-counts to enhance reliability in small datasets. 
To calculate all possible k-mers, we utilize the `product` tool from the `itertools` library.

### Discussion

In biological data, some k-mers may be underrepresented (as exemplified here by codons *AA*, *AG*, *CC*, *GC*, *GG* and *TT*). By using pseudo-counts, the `context_pseudo_probabilities()` function mitigates the impact of sparse data, providing more robust probability and paving the way for motif discovery, where accurate probability assessments are crucial.

14. Write class ```MarkovProb``` that given the $k$-th order Markov chain developed above to the constructor, its method ```probability``` computes the probability of a given input DNA sequence.

In [15]:
class MarkovProb:
    ''' Computes probabilities based on a Markov Chain '''
    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth

    def probability(self, s):
        ''' Computes the probability of a given sequence based on the Markov model '''
        if len(s) == 1:
            return self.zeroth[s]

        probabilities = 1.0
        # Handle first k-mer
        for nucleotide in range(self.k):
            # Multiply by zero-order probabilities
            probabilities *= self.zeroth[s[nucleotide]]
        # Handle k-th order probabilities
        for i in range(len(s) - self.k):
            kmer = s[i:i + self.k]
            next_nucleotide = s[i + self.k]
            probabilities *= self.kth[kmer][next_nucleotide]

        return probabilities

if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities('ATGATATCATCGACGATGTAG', k)
    zeroth = context_pseudo_probabilities('ATGATATCATCGACGATGTAG', 0)['']
    mc = MarkovProb(2, zeroth, kth)
    s = 'ATGATATCATCGACGATGTAG'
    print(f'Probability of sequence {s} is {mc.probability(s)}')

Probability of sequence ATGATATCATCGACGATGTAG is 2.831270190340017e-10


### Idea of solution

This class and method calculate the probability of observing a specific nucleotide sequence based on zeroth order and k-th order Markov probabilities. It is important to note that zeroth order and k-th order probabilities must be computed (by utilizing the `context_pseudo_probabilities()` function) prior to using this method.

### Discussion

Probabilities derived from Markov models are instrumental in predicting the likelihood of specific gene arrangements and their potential occurrences in population genetics. In this instance, even for such a short sequence, the predicted probability is expected an extremely low value. This outcome is a direct consequence of the principles of conditional probability.

With the last assignment you might end up in trouble with precision, as multiplying many small probabilities gives a really small number in the end. There is an easy fix by using so-called log-transform.
Consider computation of $P=s_1 s_2 \cdots s_n$, where $0\leq s_i\leq 1$ for each $i$. Taking logarithm in base 2 from both sides gives $\log _2 P= \log _2 (s_1 s_2 \cdots s_n)=\log_2 s_1 + \log_2 s_2 + \cdots \log s_n= \sum_{i=1}^n \log s_i$, with repeated application of the property that the logarithm of a multiplication of two numbers is the sum of logarithms of the two numbers taken separately. The results is abbreviated as log-probability.

15. Write class ```MarkovLog``` that given the $k$-th order Markov chain developed above to the constructor, its method ```log_probability``` computes the log-probability of a given input DNA sequence. Run your program with $T=$ `ATGATATCATCGACGATGTAG` and $k=2$.

In [16]:
class MarkovLog:
    ''' Computes log probabilities based on a Markov Chain '''
    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth

    def log_probability(self, s):
        ''' Computes the log probability of a given sequence '''
        if len(s) == 1:
            return np.log2(self.zeroth[s])
        # Log2 of 1 is 0 so we initialize the log probability to zero
        prob = 0.
        # Handle first k-mer
        for nucleotide in range(self.k):
            # Sum of zero-order log probabilities
            prob += np.log2(self.zeroth[s[nucleotide]])
        # Handle k-th order probabilities
        for i in range(len(s) - self.k):
            kmer = s[i:i + self.k]
            next_nucleotide = s[i + self.k]
            prob += np.log2(self.kth[kmer][next_nucleotide])

        return prob

if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities('ATGATATCATCGACGATGTAG', k)
    zeroth = context_pseudo_probabilities('ATGATATCATCGACGATGTAG', 0)['']
    mc = MarkovLog(2, zeroth, kth)
    s = 'ATGATATCATCGACGATGTAG'
    print(f'Log probability of sequence {s} is {mc.log_probability(s)}')

Log probability of sequence ATGATATCATCGACGATGTAG is -31.717831515538315


### Idea of solution

This class and method compute the log probability of a nucleotide sequence using Markov model probabilities. By converting multiplicative relationships into additive ones, this approach enhances numerical stability and improves readability. The implementation utilizes the `numpy.log2` function to perform the logarithmic calculations.

### Discussion

As expected, it can be seen that the log probability for the same sequence that was used for `Markov Prov` yields a much more significant value for human analysis and comparatives. While they won't be used any further in this notebook, log probabilities are frequently used in sequence alignment and comparative genomics to prevent underflow issues during calculations involving small probabilities, particularly in large datasets.

Finally, if you try to use the code so far for very large inputs, you might observe that the concatenation of symbols following a context occupy considerable amount of space. This is unnecessary, as we only need the frequencies.

16. Optimize the space requirement of your code from exercise 13 for the $k$-th order Markov chain by replacing the concatenations by direct computations of the frequencies. Implement this as the
  ```better_context_probabilities``` function.

In [17]:
def better_context_probabilities(s, k):
    '''
    Computes improved context probabilities using pseudo-counts.
    k-th:
    AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
    AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}, ..., }
    '''
    # To count occurrences of k-mers and next nucleotides
    context_counts = defaultdict(lambda: defaultdict(int))
    # Collect context data
    for i in range(len(s) - k):
        kmer = s[i:i + k]
        next_nucleotide = s[i + k]
        context_counts[kmer][next_nucleotide] += 1

    probabilities = {}
    # Generate all possible k-mers of length k
    all_kmers = [''.join(kmer) for kmer in product('ACGT', repeat=k)]
    for kmer in all_kmers:
        # Add pseudo count for each nucleotide base 'ACGT'
        total_follows = sum(context_counts[kmer].values()) + 4
        # Populate kmer_freqs with counts from context_counts for the current kmer, add pseudo-count then calculate probabilities
        probabilities[kmer] = {nucleotide: (context_counts[kmer][nucleotide] + 1) / total_follows for nucleotide in 'ACGT'}

    return probabilities

if __name__ == '__main__':
    k = 2
    s = 'ATGATATCATCGACGATGTAG'
    d = better_context_probabilities(s, k)
    print('\n'.join(f'{k}: {v}' for k, v in d.items()))

AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AC: {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2}
AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AT: {'A': 0.2222222222222222, 'C': 0.3333333333333333, 'G': 0.3333333333333333, 'T': 0.1111111111111111}
CA: {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4}
CC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CG: {'A': 0.5, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.16666666666666666}
CT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GA: {'A': 0.14285714285714285, 'C': 0.2857142857142857, 'G': 0.14285714285714285, 'T': 0.42857142857142855}
GC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GT: {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2}
TA: {'A': 0.16666666666666666, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.3333333333333333}
TC: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.16666666666666666}
TG: {'A': 0.3333333333333333, 'C': 0.16666666666666

### Idea of solution

We directly maintain counts of following nucleotides in a nested `defaultdict` (referred to as `context_counts`), which avoids the need to store large lists of nucleotides as strings. Pseudo counts are added directly while generating the probabilities, ensuring that we account for unseen nucleotides without the necessity of storing large suffixes. While the overall flow remains similar, this approach significantly reduces memory usage during the counting phase.

### Discussion

The `better_context_probabilities()` function computes context probabilities using a more robust approach than the `pseudo_context_probabilities()` function, ensuring greater reliability in the analysis. As expected, the output itself is identical to the latter, with *AA*, *AG*, *CC*, *GC*, *GG* and *TT* having pseudo-counts for this particular set.

While the earlier approach of explicit concatenation of symbols following a context suffered from inefficient use of space, it does have a benefit of giving another much simpler strategy to sample from the distribution:
observe that an element of the concatenation taken uniformly randomly is sampled exactly with the correct probability.

17. Revisit the solution 12 and modify it to directly sample from the concatenation of symbols following a context. The function ```np.random.choice``` may be convenient here. Implement the modified version as the new `SimpleMarkovChain` class.

In [18]:
class SimpleMarkovChain:
    ''' Generates sequences using a simple Markov Chain model '''
    def __init__(self, s, k):
        self.s = s
        self.k = k
        self.kth = better_context_probabilities(s, k)
        self.zeroth = better_context_probabilities(s, 0)['']

    def generate(self, n, seed=0):
        ''' Generates a sequence of length n based on the Markov model '''
        if n == 0:
            return ''
        if seed is not None:
            np.random.seed(seed)

        nucleotides = list('ACGT')
        p_zeroth = list(self.zeroth.values())
        start = np.random.choice(nucleotides, p=p_zeroth)
        sequence = [start]
        while n > len(sequence):
            step = ''.join(sequence[-self.k:])
            if step in self.kth:
                next_nucleotide = np.random.choice(nucleotides, p=list(self.kth[step].values()))
            else:
                next_nucleotide = np.random.choice(nucleotides, p=p_zeroth)
            sequence.append(next_nucleotide)

        return ''.join(sequence)

if __name__ == '__main__':
    k = 2
    s = 'ATGATATCATCGACGATGTAG'
    n = 10
    seed = 7
    mc = SimpleMarkovChain(s, k)
    print(mc.generate(n, seed))

ATCGTCGACG


### Idea of solution

This method generates a sequence of length **n** using a simple `Markov Chain` based on enhanced context probability distributions. It is important to note that the body of the `while` loop now operates on lists and indices rather than directly on dictionary key values, which improves efficiency in the sequence generation process.

### Discussion

The function works as expected, providing similarly distributed sequences. The generative potential of `Markov Chains` can now be explored with more complex data and probability distributions, as we will demonstrate next.

## $k$-mer index

Our $k$-th order Markov chain can now be modified to a handy index structure called $k$-mer index. This index structure associates to each $k$-mer its list of occurrence positions in DNA sequence $T$.  Given a query $k$-mer $W$, one can thus easily list all positions $i$ with  $T[i..k-1]=W$.

18. Implement function ```kmer_index``` inspired by your earlier code for the $k$-th order Markov chain. Test your program with `ATGATATCATCGACGATGTAG` and $k=2$.

In [19]:
def kmer_index(s, k):
    '''
    Returns the starting indices of all k-mers in the sequence.
    2-mer index is:
    {'AA': [], 'AC': [12], 'AG': [19], 'AT': [0, 3, 5, 8, 15], 'CA': [7], ... }
    '''
    kth = better_context_probabilities(s, k)
    k_indices = {}
    for kmer in kth.keys():
        start = 0
        indices = []
        while True:
            start = s.find(kmer, start)
            if start == -1:
                break
            indices.append(start)
            start += 1
        k_indices[kmer] = indices

    return k_indices

if __name__ == '__main__':
    k=2
    s = 'ATGATATCATCGACGATGTAG'
    print('Using string:')
    print(s)
    print(''.join([str(i%10) for i in range(len(s))]))
    print(f'\n{k}-mer index is:')
    d = kmer_index(s, k)
    print(dict(d))

Using string:
ATGATATCATCGACGATGTAG
012345678901234567890

2-mer index is:
{'AA': [], 'AC': [12], 'AG': [19], 'AT': [0, 3, 5, 8, 15], 'CA': [7], 'CC': [], 'CG': [10, 13], 'CT': [], 'GA': [2, 11, 14], 'GC': [], 'GG': [], 'GT': [17], 'TA': [4, 18], 'TC': [6, 9], 'TG': [1, 16], 'TT': []}


### Idea of solution

The `kmer_index()` function returns the start indices of each k-mer found in the given sequence. This function is useful for identifying the positions of specific k-mers within a larger sequence.

### Discussion

The output works as expected. Although the `kmer_index()` function will not be used further in this notebook, it still provides valuable insights about k-mer frequencies and positions. Possible uses can be the identification of motifs as well as the tracking and comparison of sequence elements across different datasets.

## Comparison of probability distributions

Now that we know how to learn probability distributions from data, we might want to compare two such distributions, for example, to test if our programs work as intended.

Let $P=\{p_1,p_2,\ldots, p_n\}$ and $Q=\{q_1,q_2,\ldots, q_n\}$ be two probability distributions for the same set of $n$ events. This means $\sum_{i=1}^n p_i=\sum_{i=1}^n q_i=1$, $0\leq p_j \leq 1$, and $0\leq q_j \leq 1$ for each event $j$.

*Kullback-Leibler divergence* is a measure $d()$ for the *relative entropy* of $P$ with respect to $Q$ defined as
$d(P||Q)=\sum_{i=1}^n p_i \log\frac{p_i}{q_i}$.


This measure is always non-negative, and 0 only when $P=Q$. It can be interpreted as the gain of knowing $Q$ to encode $P$. Note that this measure is not symmetric.

19. Write function ```kullback_leibler``` to compute $d(P||Q)$. Test your solution by generating a random RNA sequence
  encoding the input protein sequence according to the input codon adaptation probabilities.
  Then you should learn the codon adaptation probabilities from the RNA sequence you generated.
  Then try the same with uniformly random RNA sequences (which don't have to encode any
  specific protein sequence). Compute the relative entropies between the
  three distribution (original, predicted, uniform) and you should observe a clear difference.
  Because $d(P||Q)$ is not symmetric, you can either print both $d(P||Q)$ and $d(Q||P)$,
  or their average.
  
  This problem may be fairly tricky. Only the `kullback_leibler` function is automatically tested. The codon probabilities is probably a useful helper function. The main guarded section can be completed by filling out the `pass` sections using tooling from previous parts and fixing the *placeholder* lines.

In [20]:
def codon_prob_from_sequence(rna):
    '''
    Given an RNA sequence, simply calculates the proability of
    all 3-mers empirically based on the sequence
    '''
    k = 3
    codon_dict = get_dict()
    codon_counts = defaultdict(int) 
    probability_dict = {}

    for i in range(0, len(rna) - k, k): # Collect context data
        kmer = rna[i:i + k]
        codon_counts[kmer] += 1

    for codon in codon_dict.keys():
        count = codon_counts.get(codon, 0)
        probability_dict[codon] = (count + 1)/ (len(rna)/3 + 64) #Add pseudo counts

    return probability_dict

def codon_prob_from_table():
    '''
    Calculate probabilities for all codons based codon usage table
    '''
    html_content = get_table()
    pre_content = re.search(r'<pre>(.*?)</pre>', html_content, re.DOTALL)  # Find all codon lines in the <pre> tag
    if pre_content:
        pre_data = pre_content.group(1)
        # Initialize dictionaries to hold counts and probabilities
        codon_counts = {}
        total_count = 0
        probability_dict = {}
        # Find all matches for codons, amino acids, and counts
        pattern = r'([AUGC]{3})\s+([A-Z\*])\s+.*?\s*\(\s*(\d+)\s*\)'
        matches = re.findall(pattern, pre_data)

        # Calculate the counts
        for codon, amino_acid, count in matches:  
            codon_counts[codon] = int(count)
            total_count += int(count)

        # Calculate probabilities
        for codon, count in codon_counts.items():
            probability_dict[codon] = (count + 1 ) / (total_count + 64) #Add pseudo counts

        return probability_dict

def kullback_leibler(p, q):
    '''
    Computes Kullback-Leibler divergence between two distributions.
    Both p and q must be dictionaries from events to probabilities.
    The divergence is defined only when q[event] == 0 implies p[event] == 0.
    '''
    kl_divergence = 0
    epsilon = 1e-6
    for codon in p.keys():
        kl_divergence += p[codon] * np.log2((p[codon] + epsilon) / (q[codon] + epsilon))
    return kl_divergence

if __name__ == '__main__':
    aas = list('*ACDEFGHIKLMNPQRSTVWY') # List of amino acids
    n = 10000
    # generate a random protein and some associated rna
    protein = "".join(choice(aas, n))
    rna = ProteinToRandomRNA().convert(protein)
    # Maybe check that converting back to protein results in the same sequence
    print('RNA to Protein succesful:', rna_to_prot(rna) == protein)
    # Calculate codon probabilities of the rna sequence  
    cp_predicted = codon_prob_from_sequence(rna)
    # Calculate codon probabilities based on the codon usage table
    cp_orig = codon_prob_from_table() 
    # Create a completely random RNA sequence and get the codon probabilities
    # codons = [''.join(codon) for codon in product('ACGU', repeat = 3)]
    # random_rna = ''.join(np.random.choice(codons, n))
    random_rna = ''.join(np.random.choice(list('ACGU'), 3*n))
    cp_uniform = codon_prob_from_sequence(random_rna)

    print('d(original || predicted) =', kullback_leibler(cp_orig, cp_predicted))
    print('d(predicted || original) =', kullback_leibler(cp_predicted, cp_orig))
    print('average =', (kullback_leibler(cp_orig, cp_predicted) + kullback_leibler(cp_predicted, cp_orig))/2)
    print()
    print('d(original || uniform) =', kullback_leibler(cp_orig, cp_uniform))
    print('d(uniform || original) =', kullback_leibler(cp_uniform, cp_orig))
    print('average =', (kullback_leibler(cp_orig, cp_uniform) + kullback_leibler(cp_uniform, cp_orig))/2)
    print()
    print('d(predicted || uniform) =', kullback_leibler(cp_predicted, cp_uniform))
    print('d(uniform || predicted) =', kullback_leibler(cp_uniform, cp_predicted))
    print('average =', (kullback_leibler(cp_predicted, cp_uniform) + kullback_leibler(cp_uniform, cp_predicted))/2)

RNA to Protein succesful: True
d(original || predicted) = 0.18330779879626566
d(predicted || original) = 0.26109801878489997
average = 0.2222029087905828

d(original || uniform) = 0.2076934287813098
d(uniform || original) = 0.2760279728471799
average = 0.24186070081424485

d(predicted || uniform) = 0.2463070478166842
d(uniform || predicted) = 0.27493106443980964
average = 0.26061905612824693


### Idea of solution

The `codon_prob_from_sequence()` function calculates the empirical probabilities of each 3-mer found in an RNA sequence, essentially recycling previous functionalities. Besides, the `codon_prob_from_table()` function calculates the probabilities for all codons based on the codon usage table. This time, the probability of each codon is calculated by dividing the number of that codon by the total codon number. This function differs from previously-defined 'def get_probability_dict)' calculate probabilities for all codons with respect to their amino acid.

Subsequently, the `kullback_leibler()` function computes the Kullback-Leibler divergence between two probability distributions to quantify the difference between them. This is a straightforward implementation suitable for this notebook. The addition of `epsilon` makes the `p[codon]` and `q[codon]` have a non-zero value. This ensures the integrity of the divergence calculation without changing the final result much.

### Discussion

The Kullback-Leibler (KL) divergence values show that the predicted distribution closely approximates the original distribution. For example, $D_{\text{KL}}(\text{original} \| \text{predicted}) = 0.1833$ and $D_{\text{KL}}(\text{predicted} \| \text{original}) = 0.2620$, with an average of $0.2222$. These results indicate that the model effectively captures the characteristics of the data, with a small divergence between the predicted and original distributions.

Both the original and predicted distributions do not differ significantly from the uniform distribution. For instance, $D_{\text{KL}}(\text{original} \| \text{uniform}) = 0.2077$ and $D_{\text{KL}}(\text{uniform} \| \text{original}) = 0.2760$, with an average divergence of $0.2419$. This suggests that the original distribution is relatively close to a uniform spread.

Similarly, the predicted distribution aligns well with the uniform distribution, as evidenced by $D_{\text{KL}}(\text{predicted} \| \text{uniform}) = 0.2463$ and $D_{\text{KL}}(\text{uniform} \| \text{predicted}) = 0.2749$, yielding an average of $0.2606$. These values reinforce the observation that the predicted distribution, like the original, shows minimal divergence from uniformity.

The asymmetry in KL divergence is apparent, with divergence values differing depending on the order of comparison. Overall, the results highlight the predictive model's capability to approximate the original distribution while showing that both the original and predicted distributions are relatively close to uniform distribution.

## Stationary and equilibrium distributions (extra)

Let us consider a Markov chain of order one on the set of nucleotides.
Its transition probabilities can be expressed as a $4 \times 4$ matrix
$P=(p_{ij})$, where the element $p_{ij}$ gives the probability of the $j$th nucleotide
on the condition the previous nucleotide was the $i$th. An example of a transition matrix
is

\begin{array}{l|rrrr}
 &     A &    C &     G &    T \\
\hline
A &  0.30 &  0.0 &  0.70 &  0.0 \\
C &  0.00 &  0.4 &  0.00 &  0.6 \\
G &  0.35 &  0.0 &  0.65 &  0.0 \\
T &  0.00 &  0.2 &  0.00 &  0.8 \\
\end{array}.

A distribution $\pi=(\pi_1,\pi_2,\pi_3,\pi_4)$ is called *stationary*, if
$\pi = \pi P$ (the product here is matrix product).

20. Write function ```get_stationary_distributions``` that gets a transition matrix as parameter,
  and returns the list of stationary distributions. You can do this with NumPy by
  first taking transposition of both sides of the above equation to get equation
  $\pi^T = P^T \pi^T$. Using numpy.linalg.eig take all eigenvectors related to
  eigenvalue 1.0. By normalizing these vectors to sum up to one get the stationary distributions
  of the original transition matrix. In the ```main``` function print the stationary distributions
  of the above transition matrix.

In [21]:
def get_stationary_distributions(transition):
    '''
    For a transition matrix of a degree one Markov chain,
    it returns a list of stationary distributions.
    '''
    eigenval, eigenvec = np.linalg.eig(transition.T)
    eigen_one = np.isclose(eigenval, 1, atol=1e-6)
    # If no eigenvalue 1, return empty array
    if not np.any(eigen_one):
        return []
    # Select eigenvectors for eigenvalue 1
    selected = eigenvec[:, eigen_one]
    # Normalize probabilities
    stationary_distributions = selected / np.sum(selected, axis=0)
    
    return stationary_distributions.T
    
if __name__ == "__main__":
    transition = np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    for p in get_stationary_distributions(transition):
        print(', '.join(f'{pv:+.3f}' for pv in p))

+0.333, +0.000, +0.667, +0.000
-0.000, +0.250, -0.000, +0.750


### Idea of solution

The `get_stationary_distributions()` function derives the stationary distributions of a Markov chain represented by a transition matrix. It utilizes linear algebra concepts, specifically eigenvalues and eigenvectors (through NumPy `linalg.eig()`), to manipulate the transition matrix and obtain its stationary distributions.

### Discussion

Eigenvectors and eigenvalues provide a powerful framework for understanding the behavior of a matrix transformation. Eigenvectors define invariant directions in the data space, and eigenvalues quantify the scaling effect along those directions, enabling an alternative coordinate system that preserves the system's internal coherence. The eigenvector associated with an eigenvalue of 1 is particularly significant, as it represents a stationary state; when the system starts in this distribution, it remains unchanged under repeated applications of the transformation. This concept is essential in various fields, such as Markov chains and dynamical systems, where it explains equilibrium states or steady-state behavior.

The resulting distributions indicate the long-term probabilities of being in each state of the Markov chain. For example, in the **first distribution**, the corresponding eigenvectors indicate that there is a $33.3$% chance of being in state 1 (*A*) and a $66.7$% chance of being in state 3 (*G*) in the long run. In the **second distribution**,  the corresponding eigenvectors indicate that there is a $25$% chance of being in state 2 (*C*) and a $75$% chance of being in state 4 (*T*) in the long run. This is coherent with the provided transition matrix, which is heavily skewed to either one cluster of nucleotides or the other (*A* and *G*, or *C* and *T*).

21. Implement the `kl_divergence` function below so that the main guarded code runs properly. Using your modified Markov chain generator generate a nucleotide sequence $s$ of length $10\;000$. Choose prefixes of $s$ of lengths $1, 10, 100, 1000$, and $10\;000$. For each of these prefixes find out their nucleotide distribution (of order 0) using your earlier tool. Use 1 as the pseudo count. Then, for each prefix, compute the KL divergence between the initial distribution and the normalized nucleotide distribution.

In [22]:
class ModdedMarkovChain:
    ''' Generates sequences using a modified Markov Chain model '''
    def __init__(self, initial, transition):
        self.initial = initial
        self.transition = transition

    def generate(self, n, seed=None):
        ''' Generates a sequence of length n using the modified Markov model '''
        if n == 0:
            return ''
        if seed is not None:
            np.random.seed(seed)

        nucleotide_map = dict(zip(list('ACGT'), range(4)))
        start = np.random.choice(list('ACGT'), p=self.initial)
        sequence = [start]
        while n > len(sequence):
            next_nucleotide = np.random.choice(list('ACGT'), p=self.transition[nucleotide_map[sequence[-1]], :])
            sequence.append(next_nucleotide)

        return ''.join(sequence)

def nucleotide_distribution(prefix):
    ''' Calculate the nucleotide distribution for a given prefix. '''
    total = len(prefix) + 4 # Add 4 for pseudo-counts
    
    for nucleotide in 'ACGT':
        distribution[nucleotide] = (prefix.count(nucleotide) + 1) / total # Add pseudo-counts
    return distribution

def kl_divergences(initial, transition):
    '''
    Calculates Kullback-Leibler divergences between empirical distributions
    generated using a Markov model seeded with an initial distribution and a transition
    matrix, and the initial distribution.
    Sequences of length [1, 10, 100, 1000, 10000] are generated.
    '''
    n = 10000
    seq = ModdedMarkovChain(initial, transition).generate(n)

    # Create the initial distribution 
    p = {nucleotide: prob for nucleotide, prob in zip('ACGT', initial)}

    results = []
    for length in [1, 10, 100, 1000, 10000]:
        prefix = seq[:length]
        q = nucleotide_distribution(prefix)
        kl_divergence = kullback_leibler(p, q)
        results.append((length, kl_divergence))

    return results

if __name__ == "__main__":
    transition = np.array([[0.3, 0, 0.7, 0],
                       [0, 0.4, 0, 0.6],
                       [0.35, 0, 0.65, 0],
                       [0, 0.2, 0, 0.8]])
    print('Transition probabilities are:')
    print(transition)
    stationary_distributions = get_stationary_distributions(transition)
    print('Stationary distributions:')
    print(np.stack(stationary_distributions))
    initial = stationary_distributions[1]
    print('Using [{}] as initial distribution\n'.format(', '.join(f'{v:.2f}' for v in initial)))
    results = kl_divergences(initial, transition)
    for prefix_length, divergence in results: # iterate on prefix lengths in order (1, 10, 100...)
        print('KL divergence of stationary distribution prefix ' \
              'of length {:5d} is {:.8f}'.format(prefix_length, divergence))

Transition probabilities are:
[[0.3  0.   0.7  0.  ]
 [0.   0.4  0.   0.6 ]
 [0.35 0.   0.65 0.  ]
 [0.   0.2  0.   0.8 ]]
Stationary distributions:
[[ 0.33333333  0.          0.66666667  0.        ]
 [-0.          0.25       -0.          0.75      ]]
Using [-0.00, 0.25, -0.00, 0.75] as initial distribution

KL divergence of stationary distribution prefix of length     1 is 0.76064835
KL divergence of stationary distribution prefix of length    10 is 0.24607653
KL divergence of stationary distribution prefix of length   100 is 0.03043537
KL divergence of stationary distribution prefix of length  1000 is 0.00309521
KL divergence of stationary distribution prefix of length 10000 is 0.00045159


### Idea of solution

For this exercise, it was necessary to modify the previous `MarkovChain` class to generate a sequence using an initial distribution and transition matrix, rather than relying on zeroth and k-th order probabilities. The `nucleotide_distribution()` function calculates the distribution of nucleotides within a given sequence prefix for frequency analysis. Subsequently, the `kl_divergences()` function computes KL divergences between empirical distributions derived from sequences generated by a Markov model and an equilibrium distribution across various lengths.

### Discussion

The KL divergence results provide insight into how closely the initial distribution (*A*: 0, *C*: 0.25, *G*: 0, *T*: 0.75) approximates the stationary distribution over time. The decreasing KL divergence values indicate that as the length of the sequence increases, the initial distribution converges towards the stationary distribution. Specifically:

* At a prefix length of 1, the KL divergence is $0.760$, suggesting a significant difference between the initial and stationary distributions.
* By a prefix length of 10, the divergence decreases to $0.246$, indicating a closer alignment.
* At lengths of 100, 1000, and 10,000, the KL divergence continues to decrease significantly, reaching values as low as $0.00045$, which implies that the initial distribution closely resembles the stationary distribution after many transitions.

In summary, the KL divergence analysis shows that the Markov chain is likely to stabilize, with the initial distribution converging towards the stationary distribution as the number of transitions increases. This behavior highlights the system's tendency to reach a steady state, which is crucial for understanding its long-term dynamics.

22. Implement the following in the ```main``` function.
Find the stationary distribution for the following transition matrix:  

\begin{array}{ l | r r r r}
 & A &     C &     G &     T \\
\hline
A &  0.30 &  0.10 &  0.50 &  0.10 \\
C &  0.20 &  0.30 &  0.15 &  0.35 \\
G &  0.25 &  0.15 &  0.20 &  0.40 \\
T &  0.35 &  0.20 &  0.40 &  0.05 \\
\end{array}

Since there is only one stationary distribution, it is called the *equilibrium distribution*.
Choose randomly two nucleotide distributions. You can take these from your sleeve or
sample them from the Dirichlet distribution. Then for each of these distributions
as the initial distribution of the Markov chain, repeat the above experiment.

The `main` function should return tuples, where the first element is the (random) initial distribution and the second element contains the results as a list of tuples where the first element is the kl divergence and the second element the empirical nucleotide distribution, for the different prefix lengths.

The state distribution should converge to the equilibrium distribution no matter how we
start the Markov chain! That is the last line of the tables should have KL-divergence very close to $0$ and an empirical distribution very close to the equilibrium distribution.


In [23]:
def generate_random_alpha(num_params, low=1, high=20):
    ''' Generate a random set of alpha parameters for the Dirichlet distribution '''
    return np.random.randint(low, high, size=num_params)

def main(transition, equilibrium_distribution):
    ''' Main function to generate distributions and results as an iterable '''
    results = []
    # Ensure reproductibility for KL divergence
    np.random.seed(0)
    # Generate two different initial distributions
    for _ in range(2):
        alphas = generate_random_alpha(4, low=1, high=20)
        initial = np.random.dirichlet(alphas, size=1)[0]
        # Number of samples
        n = 10000
        seq = ModdedMarkovChain(initial, transition).generate(n)
        # Create the initial distribution
        p = {nucleotide: prob for nucleotide, prob in zip('ACGT', equilibrium_distribution)}

        for length in [1, 10, 100, 1000, 10000]:
            prefix = seq[:length]
            q = nucleotide_distribution(prefix)
            kl_divergence = kullback_leibler(p, q)  # Compare with equilibrium distribution
            results.append((kl_divergence, [probability for probability in q.values()]))

        # Yield the initial distribution and results as a single iterable
        yield initial, results

if __name__ == "__main__":
    transition = np.array([[0.3, 0.1, 0.5, 0.1],
                           [0.2, 0.3, 0.15, 0.35],
                           [0.25, 0.15, 0.2, 0.4],
                           [0.35, 0.2, 0.4, 0.05]])
    
    print('Transition probabilities are:', transition, sep='\n')
    stationary_distributions = get_stationary_distributions(transition)
    # Uncomment the below line to check that there actually is only one stationary distribution
    # assert len(stationary_distributions) == 1
    equilibrium_distribution = stationary_distributions[0]
    print('Equilibrium distribution:')
    print(equilibrium_distribution)
    for initial_distribution, results in main(transition, equilibrium_distribution):
        print('\nUsing {} as initial distribution:'.format(initial_distribution))
        print('kl-divergence   empirical distribution')
        print('\n'.join('{:.5f}   {}'.format(di, kl) for di, kl in results))

Transition probabilities are:
[[0.3  0.1  0.5  0.1 ]
 [0.2  0.3  0.15 0.35]
 [0.25 0.15 0.2  0.4 ]
 [0.35 0.2  0.4  0.05]]
Equilibrium distribution:
[0.27803345 0.17353238 0.32035021 0.22808396]

Using [0.39399635 0.48660671 0.01076917 0.10862777] as initial distribution:
kl-divergence   empirical distribution
0.18403   [0.2, 0.4, 0.2, 0.2]
0.13014   [0.21428571428571427, 0.35714285714285715, 0.21428571428571427, 0.21428571428571427]
0.01497   [0.22115384615384615, 0.18269230769230768, 0.3269230769230769, 0.2692307692307692]
0.00181   [0.2559760956175299, 0.17828685258964144, 0.3296812749003984, 0.2360557768924303]
0.00026   [0.27658936425429825, 0.16873250699720113, 0.328468612554978, 0.2262095161935226]

Using [0.38030874 0.24251191 0.25314513 0.12403423] as initial distribution:
kl-divergence   empirical distribution
0.18403   [0.2, 0.4, 0.2, 0.2]
0.13014   [0.21428571428571427, 0.35714285714285715, 0.21428571428571427, 0.21428571428571427]
0.01497   [0.22115384615384615, 0.18269230

### Idea of solution

The `generate_random_alpha()` function generates a uniformly random set of integer alpha parameters suitable for a Dirichlet distribution within specified bounds. The `main()` function serves as the control loop for generating and iteratively sampling sequences from a Markov chain, while calculating Kullback-Leibler divergences to assess the similarities between generated and equilibrium distributions. A random seed is set at the beginning to ensure the reproducibility of the KL results.

### Discussion

The transition matrix describes the probabilities of transitioning between the four nucleotide states: **A**, **C**, **G**, and **T**. Each row of the matrix sums to 1. The equilibrium distribution, which is the stationary distribution for this Markov chain, is calculated to be approximately:

- **A**: $27.8$%
- **C**: $17.4$%
- **G**: $32.0$%
- **T**: $22.8$%

This distribution indicates the long-term probabilities of being in each state, regardless of the initial distribution. To test the convergence of the Markov chain to this equilibrium distribution, two random initial distributions were generated using the Dirichlet distribution. This distribution is flexible enough to model probabilities that sum to one (such as proportions of nucleotide frequencies). By generating initial conditions with this distribution and measuring KL divergence at various lengths, we can gain valuable insights into the behavior of our Markov chain models.

The results show how the state distribution evolves over time with a sequence of 10,000 samples for the **Initial Distribution 1**. The KL divergence begins at $0.184$, straying moderately from the equilibrium distribution. As the sequence length increases, the KL divergence decreases, reaching values as low as $0.00026$, demonstrating that the empirical distribution converges closely to the equilibrium distribution. An essentially identical process was observed for the **Initial Distribution 2**, with the final KL divergence reaching $0.00034$, and the empirical distribution also approximating the equilibrium distribution.

| Initial Distribution | Base State | Initial Probability | Final Probability | KL Divergence       |
|---------------------|------------|---------------------|-------------------|----------------------|
| 1                   | A          | 39.4%               | 27.7%             | 0.184                |
|                     | C          | 48.7%               | 16.9%             |                      |
|                     | G          |  1.1%               | 32.8%             |                      |
|                     | T          | 10.9%               | 22.6%             |                      |
|                     |            |                     |                   |                      |
| 2                   | A          | 38.0%               | 28.5%             | 0.1840               |
|                     | C          | 24.3%               | 16.9%             |                      |
|                     | G          | 25.3%               | 32.3%             |                      |
|                     | T          | 12.4%               | 22.2%             |                      |


The results confirm that regardless of the initial distribution chosen, the Markov chain converges to the equilibrium distribution. The KL divergence values consistently decrease, indicating that the empirical distributions become increasingly similar to the equilibrium distribution as the number of samples increases. This behavior illustrates the fundamental property of Markov chains: they tend to reach a steady state over time, which is crucial for understanding their long-term dynamics.